In [1]:
import pydot
import pyparsing
import numpy as np
import csv
import os

import pandas as pd

### Functions

In [2]:
def getRoots(aNeigh): #parsing jd call graphs as an list
    connectedList=[]
    for i in range(len(aNeigh)):
        def findRoot(aNode,aRoot):
            while aNode != aRoot[aNode][0]:
                aNode = aRoot[aNode][0]
            return (aNode,aRoot[aNode][1])
        myRoot = {} 
        for myNode in aNeigh[i].keys():
            myRoot[myNode] = (myNode,0)  
        for myI in aNeigh[i]: 
            for myJ in aNeigh[i][myI]: 
                (myRoot_myI,myDepthMyI) = findRoot(myI,myRoot) 
                (myRoot_myJ,myDepthMyJ) = findRoot(myJ,myRoot) 
                if myRoot_myI != myRoot_myJ: 
                    myMin = myRoot_myI
                    myMax = myRoot_myJ 
                    if  myDepthMyI > myDepthMyJ: 
                        myMin = myRoot_myJ
                        myMax = myRoot_myI
                    myRoot[myMax] = (myMax,max(myRoot[myMin][1]+1,myRoot[myMax][1]))
                    myRoot[myMin] = (myRoot[myMax][0],-1) 
        myToRet = {}
        for myI in aNeigh[i]: 
            if myRoot[myI][0] == myI:
                myToRet[myI] = []
        for myI in aNeigh[i]: 
            myToRet[findRoot(myI,myRoot)[0]].append(myI)
        connectedList.append(myToRet)
    return connectedList

In [3]:
def getGraphlist(nodeList,edgeList): #getting graphs as a list
    graphList=list()
    for i in range(len(nodeList)):
        graph={}
        for j in range(len(nodeList[i])):
            nodeName=nodeList[i][j].get_name()[1:-1]
            hashtagIndex=nodeName.find("#")
            if(hashtagIndex!=-1):
                nodeName=nodeName[hashtagIndex+1:]
            someList=[]
            for k in range(len(edgeList[i])):
                if(edgeList[i][k][0]==nodeName):
                   someList.append(edgeList[i][k][1])
            graph[nodeName]=someList
        graph=findKeyAdd(graph)
        graphList.append(graph)
    
    
    return graphList

In [4]:
def findKeyAdd(graph):
    realKeyList=[]
    for key in graph:
        if(graph[key]!=[]):
            for i in range(len(graph[key])):
                realKeyList.append(graph[key][i])
    for i in range(len(realKeyList)):
        if realKeyList[i] not in graph:
            graph[realKeyList[i]]=[]
            
    return graph        

In [5]:
def matchList_writer(csv_file,csv_columns,dict_data): #writing to csv
    with open(csv_file,'w') as csvFile:
        writer=csv.DictWriter(csvFile, fieldnames=csv_columns)
        writer.writeheader()
        for data in dict_data:
            writer.writerow(data)
    return

In [7]:
def EdgeCheck(someString):  #checking if the edge is dotted or bold, it returns required index substraction
    if(someString[len(someString)-4]=="e"):#dotted
        return -18
    elif(someString[len(someString)-4]=="l"):#bold
        return -16
    else:
        return -2

In [8]:
def getEdgeList(edgeList): #getting edge list for each project in a list
    fullEdgeList=list()
    for j in range(len(edgeList)):
        edge_list=list()
        
        for i in range(len(edgeList[j])):
            pointList=list()
            edgeIndex=edgeList[j][i].to_string().find("->")
            
            point1=edgeList[j][i].to_string()[1:edgeIndex-1]
            hashtagIndex1=point1.find("#")
            point1=point1[hashtagIndex1+1:-1]

            
            s=EdgeCheck(edgeList[j][i].to_string())
            point2=edgeList[j][i].to_string()[edgeIndex+1:s]
            hashtagIndex2=point2.find("#")
            point2=point2[hashtagIndex2+1:]


            pointList.append(point1)
            pointList.append(point2)

            edge_list.append(pointList)

        fullEdgeList.append(edge_list)
    
    return fullEdgeList

    


    
    

In [6]:
def find_dstar(matrixpath): #calculating dstar for each project
    f=open(matrixpath,"r") 
    #WARNING: In our gzoltar projects matrix lines ended with + or -, yet our advisor's matrix files ended with
    # 0 or 1 We did our dstar calculation according to our advisor's matrix file, by changing 0 and 1 to - and +
    # respectively
    array=[]
    for line in f:
        if(line!='\n'):
            r=line.replace(" ", "")
            r=r.replace("\t","")
            if r.strip().endswith("1"):
                r = r.strip()[:-1] + "+"
            elif r.strip().endswith("0"):
                r = r.strip()[:-1] + "-"
            array.append(r.strip())


    row_length=len(array)
    column_length=len(array[0])


    Ncf=0
    Nuf=0
    Ncs=0
    dstarArray=[]
    for i in range(0,column_length-1):
        for j in range(0,row_length):
            isPassed=array[j][column_length-1]
            if isPassed == '-' and array[j][i]=='1':
                Ncf=Ncf+1

            elif isPassed=='-' and array[j][i]=='0':
                Nuf=Nuf+1
            elif isPassed=='+' and array[j][i]=='1':
                Ncs=Ncf+1
        if (Nuf+Ncs==0):
            dstarArray.append(-1)
        else:    
            dstar=(Ncf/(Nuf+Ncs)) #change ncf's power value to get different target values   
            dstarArray.append(dstar)
    return dstarArray

In [7]:
def findRanking(array): #finds ranking between each methods
    _,id = np.unique(np.asarray(array),return_inverse=True)
    result_array = (id.max() - id + 1).reshape(np.asarray(array).shape)
    return result_array

In [16]:
def get_suspiciousness_and_rank(gzoltarpath,projectName,methodNumberList,lineList,ccList,range1):
    matchList=[]
    
    for i in range(len(methodNumberList)):
        matrixPath=""
        matrixPath=gzoltarpath+"/"+projectName+"/"+str(i+1+range1)+"/matrix"
        if(lineList[i]!=0):
            suspiciousnessArray=find_dstar(matrixPath)#change according to your Suspiciousness finding method
            
            rankingArray=findRanking(suspiciousnessArray)
        yetAnotherList=[]
        
        
        maxSusp=-1
        
        if(methodNumberList[i]!=[] and lineList[i]!=0):
            
            for j in range(len(methodNumberList[i])):

                methodNumber=methodNumberList[i][j]-1
                
                if(suspiciousnessArray[methodNumber]>maxSusp): #there may be  more than one faulty method
                    maxSusp=suspiciousnessArray[methodNumber]
                    d={}
                    d['methodNumber']=methodNumber
                    d['suspiciousness']=suspiciousnessArray[methodNumber]
                    d['ranking']=rankingArray[methodNumber]
                    d['percentage']=(rankingArray[methodNumber])/lineList[i] #we normalize each data since each version doesn't have same numbers of method
                    if(projectName!="Math"):#please delete this if statement if you have math jdcallgraphs
                        d['CyclomaticComplexity']=ccList[i]
                    
                    
                            
                            
                    

        
            matchList.append(d)
        else:
            d={}
            d['methodNumber']=""
            d['suspiciousness']=""
            d['ranking']=""
            d['percentage']=""
            if(projectName!="Math"):#please delete this if statement if you have math jdcallgraphs
                d['CyclomaticComplexity']=""
            matchList.append(d)
    return matchList        
    

In [17]:
def findBugs(subNode,color): #get buggy lines from graphs
    lineList=list()
    lineNumberList=list()
    for node in subNode:
        
        nodeColor=node.get_fillcolor()
        if(nodeColor.find(color)!=-1):
            nodeName=node.get_name()
            hashtagIndex=nodeName.find("#")
            if(hashtagIndex!=-1):
                #lineNumber=nodeName[hashtagIndex+1:-1]
                #lineNumberList.append(lineNumber)
                lineList.append(nodeName[1:-1])
    
    #return lineNumberList
    return lineList

In [18]:
def getNodeAndEdgeList(projectName,path,range1,range2): # get node and edge list from dot files for all versions
    edgeList=list()
    nodeList=list()
    
    
    for i in range(range1,range2):
        projectPath=""
        if((projectName!="Chart" or i!=3)):
            projectPath=path+projectName+"/"+str(i+1)+".dot"
            graph = pydot.graph_from_dot_file(projectPath)
            edgeList.append(graph[0].get_edge_list())

            nodeList.append(graph[0].get_nodes())
        else:
            
            edgeList.append([])
            nodeList.append([])
    return nodeList,edgeList

In [19]:
def findBuggyLines(nodeList): #find buggy nodes in a graph for all versions
    bugList=list()
    for subNode in nodeList:
        lineNumberList=findBugs(subNode,"red")
        if lineNumberList:
            

            bugList.append(lineNumberList)        
        else:
            bugList.append([]) 
      #      lineNumberList=findBugs(subNode,"orange")
      #      if lineNumberList:
      #          bugList.append(lineNumberList)        
      #      else:
      #          lineNumberList=findBugs(subNode,"yellow")
      #          bugList.append(lineNumberList)
    return bugList        



   

In [20]:
def findMethodNumber(gzoltarpath,projectName,bugList,range1): #finding method line from spectra
    methodNumberList=list()
    lineList=list()
    for i in range(len(bugList)):
        spectraPath=""
        spectraPath=gzoltarpath+"/"+projectName+"/"+str(i+1+range1)+"/spectra"
        
        lineNumberList=list()
        k=1
        with open(spectraPath, "r") as ins:
            
            for line in ins:
                
                for j in range(len(bugList[i])):
                    if(bugList[i][j]==line[:-1]):
                        
                        lineNumberList.append(k)
                k=k+1
        lineList.append(k-1)
        methodNumberList.append(lineNumberList)
        
    return(methodNumberList,lineList)    

In [21]:
def getCyclomaticComplexity(connectedComponent,graphList,edgeList): #formula taken from wikipedia
    ccList=[]
    for i in range(len(nodeList)):
        M=len(edgeList[i])-len(graphList[i])+2*len(connectedComponent[i])
        ccList.append(M)
    
    return ccList

In [22]:
def mathBugFinder(gzoltarPath): #this is spesific for our math project, if you have proper jdcallgraphs for math please skip this
    with open("handmatched_math.txt") as f:
        content = f.readlines()
        lineList=list()
        lineNumberList=list()
        methodNumberList=list()
        i=0
        for line in content:
            if(line[0]=="M"):
                if(i!=0):
                    methodNumberList.append(lineNumberList)
                _Index=line.find("_")
                versionNumber=line[_Index+1:-2]
                lineNumberList=[]
                spectraPath=gzoltarpath+"/Math/"+versionNumber+"/spectra"
                i=versionNumber
                with open(spectraPath, "r") as ins:
                    k=0
                    for a in ins:
                        k=k+1
                    lineList.append(k)
            elif(line[0]=="o"):
                lineNumberList.append(line[:-1])
                if(i=="106"):
                    methodNumberList.append(lineNumberList)
                        

        return methodNumberList,lineList        
        

In [18]:
#change inputs according your preference
#some projects(Closure) are too large to analyze therefore we put range variables
graphPath="/home/fmert/Praktikum/graphs/" 
gzoltarPath="/home/fmert/fault-localization-data/gzoltar/gzoltars"
projectName="Lang"
range1=0
range2=65


if(projectName=="Math"): #in our experiments we didn't have jdcallgraphs of math project, you can delete this line if you have call graphs
    bugNumberList,lineList=mathBugFinder(gzoltarPath)
    methodNumberList,lineList=findMethodNumber(gzoltarPath,projectName,bugNumberList,range1)
    
    matchList=get_suspiciousness_and_rank(gzoltarPath,projectName,methodNumberList,lineList,[],range1)
else:
    nodeList,edgeList=getNodeAndEdgeList(projectName,graphPath,range1,range2)
    bugList=findBuggyLines(nodeList)
    methodNumberList,lineList=findMethodNumber(gzoltarPath,projectName,bugList,range1)
    edgeList2=getEdgeList(edgeList)
    graphList=getGraphlist(nodeList,edgeList2)
    connectedList=getRoots(graphList)
    ccList=getCyclomaticComplexity(connectedList,graphList,edgeList2)
    matchList=get_suspiciousness_and_rank(gzoltarPath,projectName,methodNumberList,lineList,ccList,range1)




#save to results as csv to current path
currentPath=os.getcwd()
csv_file=currentPath+"/"+projectName+"Ranking_"+str(range1)+"-"+str(range2)+".csv"

if(projectName!="Math"):
    csv_columns=['methodNumber','suspiciousness','ranking','percentage','CyclomaticComplexity']
else:
    csv_columns=['methodNumber','suspiciousness','ranking','percentage']

matchList_writer(csv_file,csv_columns,matchList)


        
